In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
import matplotlib as mpl


# ------------------------------------------------------------------
# Utility functions
# ------------------------------------------------------------------
def check_boundary_flux(rho, V, Pe, dx):
    """
    Return (left_flux, right_flux) to verify the zero‑flux
    (Neumann) boundary condition F = 0.
    """
    rho_minus1 = rho[1] - 2.0 * dx * Pe * V[0] * rho[0]
    rho_plus1  = rho[-2] + 2.0 * dx * Pe * V[-1] * rho[-1]

    drho_dx_left  = (rho[1]   - rho_minus1) / (2.0 * dx)
    drho_dx_right = (rho_plus1 - rho[-2])   / (2.0 * dx)

    F_left  = V[0]  * rho[0]  - (1.0 / Pe) * drho_dx_left
    F_right = V[-1] * rho[-1] - (1.0 / Pe) * drho_dx_right
    return F_left, F_right


def update_rho_fvm(rho, V, Pe, dx, dt):
    """
    One FVM step for rho:

        rho_i^{n+1} = rho_i^n - (dt/dx) · [F_{i+1/2} − F_{i−1/2}]

    where
        F = convection − (1/Pe) · diffusion

    * Convection: first‑order upwind using V_face.
    * Diffusion:  central difference.
    * Zero‑flux BC: flux[0] = flux[N] = 0.
    """
    N = len(rho)
    flux = np.zeros(N + 1)

    for i_face in range(1, N):
        V_face = 0.5 * (V[i_face - 1] + V[i_face])
        adv_flux = rho[i_face - 1] * V_face if V_face >= 0 else rho[i_face] * V_face
        diff_flux = (rho[i_face] - rho[i_face - 1]) / dx
        flux[i_face] = adv_flux - (1.0 / Pe) * diff_flux

    flux[0] = flux[N] = 0.0
    rho_new = rho - (dt / dx) * (flux[1:] - flux[:-1])
    return rho_new


# ------------------------------------------------------------------
# Global grid and physical constants
# ------------------------------------------------------------------
N        = 100
x        = np.linspace(-1.0,  1.0, N)
dx       = x[1] - x[0]
sigma    = 0.15
t_final  = 10.0

rho_0, alpha, mu_0, S, D, L = 1_000.0, 0.2, 1.0, 10.0, 0.9, 1.0


# ------------------------------------------------------------------
# Main simulation routine
# ------------------------------------------------------------------
def simulate_actomyosin(w_val, gamma_val, steps, dt, Pe):
    """
    Run the solver for given (w, gamma).
    Returns final (rho, a, V).
    """
    # initial fields
    a   = np.ones(N)
    rho = np.exp(-(x**2) / (2 * sigma**2))
    rho /= np.trapz(rho, x)          # mass = 1
    V   = np.zeros(N)

    for _ in range(steps):
        # --- solve V (Poisson‑like linear system) ---
        M = np.zeros((N, N))
        C = np.zeros(N)

        for i in range(1, N - 1):
            M[i, i - 1] = 1.0 / dx**2
            M[i, i]     = -2.0 / dx**2 - gamma_val * a[i]
            M[i, i + 1] = 1.0 / dx**2
            C[i]        = -(rho[i + 1] - rho[i - 1]) / (2.0 * dx)

        # left boundary
        M[0, 0] = -2.0 / dx**2 - gamma_val * a[0]
        M[0, 1] =  2.0 / dx**2
        d_rho_dx_left = (-3.0 * rho[0] + 4.0 * rho[1] - rho[2]) / (2.0 * dx)
        C[0] = -d_rho_dx_left - 2.0 * rho[0] / dx

        # right boundary
        M[-1, -1] = -2.0 / dx**2 - gamma_val * a[-1]
        M[-1, -2] =  2.0 / dx**2
        d_rho_dx_right = (3.0 * rho[-1] - 4.0 * rho[-2] + rho[-3]) / (2.0 * dx)
        C[-1] = -d_rho_dx_right + 2.0 * rho[-1] / dx

        V_new = spsolve(csr_matrix(M), C)

        # --- update a (upwind + reaction) ---
        a_new = a.copy()
        for i in range(1, N - 1):
            adv = V_new[i] * (a[i] - a[i - 1]) / dx if V_new[i] > 0 \
                  else V_new[i] * (a[i + 1] - a[i]) / dx
            a_new[i] = a[i] - dt * adv + dt * w_val * (1.0 - a[i])
        a_new[0] = a_new[-1] = 0.0

        # --- update rho ---
        rho_new = update_rho_fvm(rho, V_new, Pe, dx, dt)

        # overwrite
        a, rho, V = a_new, rho_new, V_new

    return rho, a, V


# ------------------------------------------------------------------
# Matplotlib global style
# ------------------------------------------------------------------
mpl.rcParams.update({
    "font.size":        12,
    "axes.labelsize":   14,
    "axes.titlesize":   14,
    "xtick.labelsize":  12,
    "ytick.labelsize":  12,
    "legend.fontsize":  12,
})


# ------------------------------------------------------------------
# Parameter scan: fixed gamma, varying k (Pe) and w
# ------------------------------------------------------------------
k_list = [0.0225, 0.045, 0.09]   # Pe ≈ 0.5, 1.0, 2.0
w_list = [0.5, 1.0, 2.0]
gamma_val = 2.0

fig, axes = plt.subplots(
    nrows=len(k_list),
    ncols=len(w_list),
    sharex=True,
    sharey=True,
    figsize=(9, 8)
)
fig.suptitle(f"Steady‑state profiles  (gamma = {gamma_val})", fontsize=16)

for i, k_val in enumerate(k_list):
    # compute Pe and stable time step
    Pe = (k_val * rho_0 * L**2 * alpha) / (mu_0 * S * D)
    dt_conv = dx / ((k_val * rho_0 * L * alpha) / (mu_0 * S))
    dt_diff = dx**2 / (2 * D)
    dt = min(dt_conv, dt_diff) / 5.0
    if dt < 1e-6:
        raise ValueError("Time step too small.")
    steps = round(t_final / dt)
    print(f"k = {k_val:.4f}  →  Pe = {Pe:.1f},  dt = {dt:.2e},  steps = {steps}")

    for j, w_val in enumerate(w_list):
        ax = axes[i, j]

        rho_f, a_f, V_f = simulate_actomyosin(w_val, gamma_val, steps, dt, Pe)

        ax.plot(x, rho_f, color="#c94733", label=r"$\rho(x)$")
        ax.plot(x, a_f,   color="#0d5b26", label=r"$a(x)$")
        ax.plot(x, V_f,   color="#FEBA4F", label=r"$V(x)$")

        ax.set_title(f"Pe = {Pe:.1f},  w = {w_val}")

        if i == len(k_list) - 1:
            ax.set_xlabel("x")
        if j == 0:
            ax.set_ylabel("Value (a.u.)")

        plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

# single legend outside the axes
handles, labels = axes[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc="lower center", ncol=3)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.savefig("scan_k_w_fixed_gamma.png", dpi=300)
plt.show()
